# MicrostructureNoise

(Available as Jupyter notebook at https://github.com/mschauer/MicrostructureNoise.jl/blob/master/example/MicrostructureNoise.ipynb)

`MicrostructureNoise` is a Julia package for Bayesian volatility estimation in presence of market microstructure noise implementing the described in our new preprint: 

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility learning under microstructure noise.

This blogpost 
gives a short introduction.


## Description

MicrostructureNoise estimates the volatility function $s$ of the stochastic differential equation

$dX_t = b(t,X_t) dt + s(t) dW_t, \quad X_0 = x_0, \quad t \in [0,T]"$

from noisy observations of its solution

$Y_i = X(t_i) + V_i, \quad 0 = t_0 < \ldots < t_n = T$

where $\{V_i\}$ denote unobservable stochastic disturbances. The method is minimalistic in its assumptions on the volatility function, which in particular can itself be a stochastic process.

The estimation methodology is intuitive to understand, given that its ingredients are well-known statistical techniques. The posterior inference is performed via the Gibbs sampler, with the Forward Filtering Backward Simulation algorithm used to reconstruct unobservable states $X(t_i)$. This relies on the Kalman filter. The unknown squared volatility function is a priori modelled as piecewise constant and is assigned the inverse Gamma Markov chain prior, which induces smoothing among adjacent pieces of the function. The picture below gives an idea of the results obtainable with the method. Depicted is a reconstruction of the volatility function from the synthetic data generated according to the classical Heston stochastic volatility model (the unobserved true squared volatility curve is plotted in red). Note that next to the point estimate (posterior mean plotted in black), the method conducts automatic uncertainty quantification via the marginal Bayesian credible band (plotted in blue).

<img src="../heston.png" width=600>

## Setup

Install `Microstructure` via the package manager.
```
Pkg.add("MicrostructureNoise")
```

In [4]:
using MicrostructureNoise, Distributions, Plots
srand(12);

# Real data example

As first example, we apply our methodology to infer volatility of the high frequency foreign exchange rate data made available by Pepperstone Limited, the London based forex broker (https://pepperstone.com/uk/client-resources/historical-tick-data). Specifically, we use the EUR/USD tick data (bid prices) for 2 March 2015. We obtain, log-transform and subsample the data and express time in milliseconds.

In [6]:
# uncomment if you do not mind to create this large file 
# Base.download("https://www.truefx.com/dev/data//2015/MARCH-2015/EURUSD-2015-03.zip","./data/EURUSD-2015-03.zip")
# run(`unzip ./data/EURUSD-2015-03.zip -d ./data`)
#dat = readcsv("../data/EURUSD-2015-03.csv")
times = map(a -> DateTime(a, "yyyymmdd H:M:S.s"), dat[1:10:130260,2])
t = Float64[1/1000*(times[i].instant.periods.value-times[1].instant.periods.value) for i in 1:length(times)]
n = length(t)-1
T = t[end]
y = log.(Float64.(dat[1:10:130260, 3]));

In [8]:
plot(t[1:10:end], y[1:10:end], label="EUR/USD")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 1.117 
 
 
 1.118 
 
 
 1.119 
 
 
 1.120 
 
 
 1.121 
 
 
 1.122 
 
 
 1.123 
 
<polyline clip-path="url(#clip6302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.8557,375.869 45.9762,354.724 46.4369,334.039 46.7297,336.337 46.8917,345.071 47.1745,334.039 47.3467,334.958 47.806,344.611 48.0812,340.934 48.4533,343.232 
 48.6438,340.934 49.1772,349.668 49.5812,343.232 50.4823,335.877 50.8095,345.071 50.9786,343.232 51.1689,337.716 51.3151,346.91 51.7941,361.619 52.0105,360.24 
 52.3095,362.079 52.9262,352.885 53.3316,367.595 53.715,365.756 54.3613,366.216 55.7689,366.216 56.8981,360.7 58.0343,351.966 58.7982,332.66 59.4814,343.232 
 60.4256,355.184 61.1027,355.184 61.9443,352.885 63.1,346.91 63.6548,356.563 64.7365,352.426 65.0937,340.015 66.0371,347.369 66.3109,347.829 66.6903,352.885 
 68.1995,344.152 69.1201,344.152 69.7394,331.281 70.3591,328.982 70.9267,319.329 71.2892,311.515 71.6137,326.224 72.1858,329.902 72.7589,326.224 73.8854,323.926 
 74.7302,315.652 75.1826,318.41 75.9258,323.007 76.6188,311.515 77.1177,317.031 77.4161,327.603 77.5983,334.498 77.9807,328.982 78.7574,337.716 79.376,344.152 
 80.5826,337.716 81.932,337.716 83.077,338.176 83.717,334.498 84.8826,330.361 85.469,320.249 85.6548,323.926 86.3448,327.144 87.2122,322.547 87.803,334.039 
 88.2053,321.628 88.7245,311.975 90.0007,324.845 93.4238,317.491 94.046,310.596 94.9035,314.733 95.7276,311.055 96.4397,313.354 96.7706,311.975 97.0047,315.192 
 97.216,309.676 97.591,305.08 97.9139,305.539 98.4131,311.975 98.7124,296.806 98.9671,301.402 99.4226,304.62 99.8987,312.894 100.586,310.136 101.501,303.241 
 102.078,298.644 103.318,302.781 103.907,315.192 104.361,320.708 105.395,319.329 106.102,320.249 107.234,317.031 108.196,319.789 109.555,323.926 111.405,324.386 
 112.602,322.087 113.033,326.684 113.918,331.74 115.379,322.087 117.626,323.007 118.908,311.055 119.773,305.539 121.232,313.354 121.769,316.112 122.337,319.789 
 123.061,312.894 123.559,303.701 124.353,310.136 125.212,316.112 126.099,307.378 126.57,291.29 127.312,305.999 128.659,297.725 130.074,309.676 131.086,302.322 
 132.373,303.241 133.543,302.322 135.372,304.16 137.507,300.023 139.921,305.08 141.622,302.322 144.592,300.943 147.367,302.322 148.237,292.669 149.012,288.991 
 149.686,296.346 150.807,297.725 151.822,293.128 152.398,291.29 152.916,293.128 153.216,292.669 154.287,293.128 155.787,289.451 156.577,292.669 157.893,292.669 
 160.467,293.588 161.429,296.806 162.904,284.395 163.446,283.475 164.6,289.911 167.412,293.128 169.244,287.612 172.197,276.121 174.118,277.959 175.223,275.661 
 176.563,280.258 178.837,280.258 179.242,270.145 180.159,270.145 180.539,272.903 182.505,274.282 183.478,277.959 184.796,272.903 186.357,271.524 188.184,283.935 
 189.241,282.556 189.966,288.532 191.228,286.233 192.87,280.717 194.067,287.153 195.076,292.209 195.55,288.532 197.773,292.669 199.162,290.83 199.755,296.806 
 200.656,295.427 201.035,304.62 201.204,301.862 201.458,302.322 201.716,313.354 201.857,306.918 201.95,306.918 202.072,307.838 202.5,312.434 203.467,321.168 
 203.964,311.975 204.484,320.708 205.173,327.603 205.607,332.66 206.383,328.523 206.7,317.031 207.412,318.41 208.011,305.539 208.886,311.055 209.472,305.08 
 210.075,307.378 210.803,311.515 212.32,315.652 212.997,315.192 213.682,321.628 214.381,316.112 214.806,322.087 215.166,330.361 215.774,319.789 216.119,311.975 
 216.529,314.733 216.79,311.055 217.144,317.95 217.565,315.192 218.113,319.789 218.699,326.224 219.092,326.224 219.364,339.555 219.586,337.716 219.864,333.579 
 220.217,344.611 220.904,334.039 221.145,323.466 221.436,332.66 221.842,330.361 222.324,317.031 222.625,311.515 223.216,314.273 223.388,304.16 223.801,293.128 
 224.113,297.265 224.346,289.911 224.662,295.886 224.838,293.588 225.22,297.725 225.343,293.128 225.515

The prior specification is done via the `Prior` struct.
Most importantly, we set the number of bins `N`.

In [92]:
prior = MicrostructureNoise.Prior(
N = 40,

α1 = 0.0,
β1 = 0.0,

αη = 0.01, 
βη = 0.01,

Πα = LogNormal(1., 0.5),
μ0 = 0.0,
C0 = 5.0
);

Sample the posterior using MCMC.

In [ ]:
α = 0.3 # Initial smoothness hyperparameter guess
σα = 0.1 # Random walk step size for smoothness hyperparameter

td, θs, ηs, αs, p = MicrostructureNoise.MCMC(prior, t, y, α, σα, 1500, printiter=500);

In [ ]:
The inline help gives a synopsis of the functio `MCMC`.

In [14]:
?MicrostructureNoise.MCMC

```
MCMC(Π::Union{Prior,Dict}, tt, yy, α0::Float64, σα, iterations; subinds = 1:1:iterations, η0::Float64 = 0.0, printiter = 100) -> td, θ, ηs, αs, pacc
```

Run the Markov Chain Monte Carlo procedure for `iterations` iterations, on data `(tt, yy)`, where `tt` are observation times and `yy` are observations, `α0` is the initial guess for the smoothing parameter `α` (necessary), `η0` is the iniat guess woth the noise covariance (optional), and `σα` is the stepsize for the random walk proposal for `α`.

Prints verbose output every `printiter` iteration.

Returns `td, θs, ηs, αs, pacc`, `td` is the time grid of the bins, `ηs`, `αs` are vectors of iterates, possible subsampled at indices `subinds`, `θs` is a Matrix with iterates of `θ` rows. `paccα` is the acceptance probability for the update step of `α`.

Keyword args `fixalpha`, `fixeta` when set to `true` allow fixing `α` and `η` at their initial values.


In [76]:
posterior = MicrostructureNoise.posterior_volatility(td, θs)

tt, mm = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_median[:])

plot(tt, mm, label="posterior median", color="dark blue")
plot!(MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qlow[:])...,
    fillrange = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qup[:])[2], 
    fillalpha = 0.2,
    alpha = 0.0,
    fillcolor="blue",
    title="Volatility estimate", label="marginal $(round(Int,posterior.qu*100))% credibility band")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 Volatility estimate 
 
<polyline clip-path="url(#clip0602)" style="stroke:#00008b; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1651,325.856 53.2577,325.856 53.2577,331.417 66.0449,331.417 66.0449,340.322 79.4441,340.322 79.4441,338.489 93.4226,338.489 93.4226,334.323 106.065,334.323 
 106.065,307.294 118.346,307.294 118.346,296.691 129.932,296.691 129.932,304.857 141.42,304.857 141.42,268.398 156.576,268.398 156.576,204.947 169.843,204.947 
 169.843,248.887 183.42,248.887 183.42,212.819 198.787,212.819 198.787,163.196 211.494,163.196 211.494,237.689 227.859,237.689 227.859,316.161 239.691,316.161 
 239.691,336.455 253.694,336.455 253.694,334.006 267.526,334.006 267.526,323.724 281.613,323.724 281.613,344.639 295.654,344.639 295.654,323.998 310.31,323.998 
 310.31,339.878 322.726,339.878 322.726,308.448 337.771,308.448 337.771,277.163 350.802,277.163 350.802,301.578 363.233,301.578 363.233,284.418 377.469,284.418 
 377.469,324.912 393.427,324.912 393.427,350.124 405.516,350.124 405.516,356.596 420.42,356.596 420.42,359.854 430.843,359.854 430.843,344.584 444.936,344.584 
 444.936,349.601 458.056,349.601 458.056,346.168 471.087,346.168 471.087,361.624 483.647,361.624 483.647,358.636 498.05,358.636 498.05,358.915 511.534,358.915 
 511.534,349.057 526.434,349.057 526.434,334.252 541.464,334.252 541.464,313.927 553.832,313.927 553.832,283.251 566.748,283.251 566.748,331.901 580.315,331.901 
 
 "/>
<polygon clip-path="url(#clip0602)" points="
39.1651,352.425 53.2577,352.425 53.2577,355.918 66.0449,355.918 66.0449,361.317 79.4441,361.317 79.4441,359.93 93.4226,359.93 93.4226,359.489 106.065,359.489 
 106.065,349.047 118.346,349.047 118.346,340.347 129.932,340.347 129.932,342.141 141.42,342.141 141.42,314.587 156.576,314.587 156.576,271.447 169.843,271.447 
 169.843,308.521 183.42,308.521 183.42,273.143 198.787,273.143 198.787,239.006 211.494,239.006 211.494,293.233 227.859,293.233 227.859,347.529 239.691,347.529 
 239.691,359.698 253.694,359.698 253.694,355.991 267.526,355.991 267.526,348.311 281.613,348.311 281.613,361.738 295.654,361.738 295.654,348.198 310.31,348.198 
 310.31,360.641 322.726,360.641 322.726,340.988 337.771,340.988 337.771,319.492 350.802,319.492 350.802,335.373 363.233,335.373 363.233,325.826 377.469,325.826 
 377.469,356.769 393.427,356.769 393.427,367.777 405.516,367.777 405.516,372.608 420.42,372.608 420.42,373.117 430.843,373.117 430.843,363.681 444.936,363.681 
 444.936,368.62 458.056,368.62 458.056,368.444 471.087,368.444 471.087,375.869 483.647,375.869 483.647,373.055 498.05,373.055 498.05,371.594 511.534,371.594 
 511.534,365.402 526.434,365.402 526.434,362.856 541.464,362.856 541.464,347.971 553.832,347.971 553.832,328.008 566.748,328.008 566.748,362.256 580.315,362.256 
 580.315,260.817 566.748,260.817 566.748,210.632 553.832,210.632 553.832,255.747 541.464,255.747 541.464,293.072 526.434,293.072 526.434,317.065 511.534,317.065 
 511.534,336.13 498.05,336.13 498.05,333.829 483.647,333.829 483.647,335.218 471.087,335.218 471.087,308.04 458.056,308.04 458.056,314.739 444.936,314.739 
 444.936,312.094 430.843,312.094 430.843,329.771 420.42,329.771 420.42,329.564 405.516,329.564 405.516,314.305 393.427,314.305 393.427,283.243 377.469,283.243 
 377.469,221.946 363.233,221.946 363.233,239.621 350.802,239.621 350.802,209.108 337.771,209.108 337.771,251.709 322.726,251.709 322.726,304.122 310.31,304.122 
 310.31,287.106 295.654,287.106 295.654,303.23 281.613,303.23 281.613,276.266 267.526,276.266 267.526,292.641 253.694,292.641 253.694,279.292 239.691,279.292 
 239.691,255.351 227.859,255.351 227.859,149.586 211.494,149.586 211.494,31.4961 198.787,31.4961 198.787,93.5998 183.42,93.5998 183.42,162.271 169.843,162.271 
 169.843,95.0625 156.576,95.0625 156.576,189.

A histogram of `sqrt.(ηs)` visualises the posterior of the observation error parameter `η`, indicating that there is indeed microstructure noise.

In [100]:
histogram(sqrt.(ηs[end÷2:end]), nbins=20, label="eta")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0054 
 
 
 0.0055 
 
 
 0.0056 
 
 
 0.0057 
 
 
 0.0058 
 
 
 0.0059 
 
 
 0.0060 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 1200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 eta

# How good does it work: Test with the Heston model

In [66]:
using Bridge
using StaticArrays
const R2 = SVector{2,Float64};

In [67]:
struct Heston <: ContinuousTimeProcess{R2}
    mu::Float64
    kappa::Float64
    theta::Float64
    sigma::Float64
end
Heston(;mu = NaN, kappa = NaN, theta = NaN, sigma = NaN) = Heston(mu, kappa, theta, sigma)
Bridge.b(s, x, P::Heston) = R2(P.mu*x[1], P.kappa*(P.theta - x[2]))
Bridge.σ(s, x, P::Heston) = Diagonal(R2(sqrt(x[2])*x[1], P.sigma*sqrt(x[2])))

In [69]:
T = 1.0
n = 4000
nf = 10
P = Heston(mu = 0.02, kappa = 7.0, theta = 0.04, sigma = 0.6)

Heston(0.02, 7.0, 0.04, 0.6)

In [71]:
tfine0 = T .* sort(rand(n*nf-1))
tfine = [0.0; tfine0; T]

is = [i <= n-1 for i in 1:length(tfine)-2]
is = [true; is[randperm(length(is))]; true]
t = tfine[is];
    

In [94]:
u = R2(1., P.theta)
W = sample(tfine, Wiener{R2}())
ρ = -0.6
map!(v->R2(v[1], ρ*v[1] + sqrt(1-ρ^2)v[2]), W.yy, W.yy)

Xfine = solve(EulerMaruyama(), u, W, P)

xtrue = log.(first.(Xfine.yy[is]))
s0(t) = sqrt(Xfine.yy[searchsortedfirst(Xfine.tt, t)][2])
y = copy(xtrue) 

η0 = 0.005^2
y .+= randn(n + 1) * sqrt(η0);


In [95]:
α = 5.0 # Initial smoothness hyperparameter guess
σα = 3.0 # Random walk step size for smoothness hyperparameter

td, θs, ηs, αs, p = MicrostructureNoise.MCMC(prior, t, y, α, σα, 10000, printiter=500);

println("acceptance probability $p")

500 	 α 10.187881706726099	 √η0.005634991273437541
1000 	 α 5.831465520763732	 √η0.005543311364152845
1500 	 α 5.360794458754045✓	 √η0.005749677152253645
2000 	 α 2.7285640597034937	 √η0.00562709451157429
2500 	 α 4.487298444059277✓	 √η0.005730362695423951
3000 	 α 13.380893637969377	 √η0.005656648736638853
3500 	 α 8.320397571179445	 √η0.00565196794128634
4000 	 α 0.6757992425579609	 √η0.005822986881662241
4500 	 α 5.538940443757637✓	 √η0.005604970191555754
5000 	 α 8.42235305332813	 √η0.005636766298155026
5500 	 α 5.13028791698054✓	 √η0.0056784335450804305
6000 	 α 3.23916778373014✓	 √η0.0057336989250119895
6500 	 α 11.733021131698784	 √η0.005711740396855054
7000 	 α 1.752030508496091	 √η0.00562159561718007
7500 	 α 2.0230285034994706	 √η0.005546244591180947
8000 	 α 4.324637180627985	 √η0.00557658511936378
8500 	 α 10.598909441219387	 √η0.005574004494519103
9000 	 α 11.124765272657715	 √η0.005718465401717607
9500 	 α 2.170631943957033	 √η0.005605559993098581
10000 	 α 5.759491981533

In [103]:
posterior = MicrostructureNoise.posterior_volatility(td, θs)

tt, mm = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_median[:])

plot(tt, mm, label="posterior median", linewidth=0.5, color="dark blue")
plot!(t[1:10:end], (s0.(t[1:10:end])).^2, label="true volatility", color="red")


plot!(MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qlow[:])...,
    fillrange = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qup[:])[2], 
    fillalpha = 0.2,
    alpha = 0.0,
    fillcolor="darkblue",
    title="Volatility estimate", label="marginal $(round(Int,posterior.qu*100))% credibility band")



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 Volatility estimate 
 
<polyline clip-path="url(#clip3002)" style="stroke:#00008b; stroke-width:0.5; stroke-opacity:1; fill:none" points="
 39.1651,292.487 53.2577,292.487 53.2577,333.308 66.0449,333.308 66.0449,350.907 79.4441,350.907 79.4441,356.635 93.4226,356.635 93.4226,343.416 106.065,343.416 
 106.065,331.758 118.346,331.758 118.346,353.973 129.932,353.973 129.932,360.908 141.42,360.908 141.42,359.752 156.576,359.752 156.576,334.329 169.843,334.329 
 169.843,322.464 183.42,322.464 183.42,302.358 198.787,302.358 198.787,294.628 211.494,294.628 211.494,297.169 227.859,297.169 227.859,319.62 239.691,319.62 
 239.691,312.254 253.694,312.254 253.694,334.179 267.526,334.179 267.526,346.82 281.613,346.82 281.613,356.027 295.654,356.027 295.654,346.268 310.31,346.268 
 310.31,342.213 322.726,342.213 322.726,326.642 337.771,326.642 337.771,273.86 350.802,273.86 350.802,254.058 363.233,254.058 363.233,205.068 377.469,205.068 
 377.469,239.736 393.427,239.736 393.427,174.295 405.516,174.295 405.516,244.054 420.42,244.054 420.42,160.63 430.843,160.63 430.843,275.08 444.936,275.08 
 444.936,296.206 458.056,296.206 458.056,280.223 471.087,280.223 471.087,293.046 483.647,293.046 483.647,328.08 498.05,328.08 498.05,323.732 511.534,323.732 
 511.534,281.782 526.434,281.782 526.434,291.589 541.464,291.589 541.464,298.222 553.832,298.222 553.832,234.863 566.748,234.863 566.748,252.705 580.315,252.705 
 
 "/>
<polyline clip-path="url(#clip3002)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1651,250.08 40.2969,264.264 41.3143,259.374 42.4056,264.11 44.3682,263.261 45.0159,289.512 46.4211,296.924 48.159,291.957 49.8945,307.927 51.5651,314.783 
 53.2577,315.718 54.3771,311.171 55.1075,310.18 57.5863,312.066 58.3752,301.904 59.6468,298.746 60.6509,295.524 61.8547,312.079 63.8783,310.061 64.8706,317.247 
 66.0449,342.153 67.4225,347.249 68.5917,340.273 69.8844,344.065 70.8397,351.914 72.3816,336.006 73.568,352.677 75.2324,355.225 76.2641,362.24 78.0786,361.828 
 79.4441,362.299 81.4816,367.823 82.4016,366.361 83.2375,371.175 85.0997,375.869 86.0585,374.569 87.794,367.121 89.0158,375.027 90.1579,375.486 91.5437,368.188 
 93.4226,357.141 95.3044,358.39 96.3766,354.819 97.5947,339.855 98.7927,344.797 99.5488,331.445 101.382,318.897 102.601,328.516 104.053,338.131 104.97,343.4 
 106.065,351.301 107.502,350.379 108.582,343.627 109.395,339.907 111.181,342.817 112.365,350.019 113.342,338.678 114.149,328.478 114.895,325.416 116.538,318.751 
 118.346,340.174 119.068,340.911 120.196,357.903 121.509,359.316 122.546,359.351 123.581,357.968 125.501,357.797 127.045,366.233 127.935,359.48 129.139,358.779 
 129.932,364.243 130.632,357.687 131.969,352.235 133.159,356.883 134.553,359.603 135.121,363.605 136.199,366.206 137.153,358.653 138.972,352.327 140.322,334.479 
 141.42,341.264 142.843,332.223 144.532,344.09 146.074,334.4 147.872,342.624 148.907,336.057 150.581,337.864 152.356,335.149 153.725,323.223 155.245,331.989 
 156.576,338.893 158.026,350.362 159.436,342.74 160.687,335.563 161.822,328.012 163.565,315.14 164.35,333.018 165.311,332.216 166.637,310.379 168.065,311.973 
 169.843,316.133 170.57,326.067 172.071,327.345 173.787,312.106 174.793,316.974 175.807,314.563 177.05,334.118 177.811,328.399 179.433,330.128 181.08,318.013 
 183.42,301.195 184.301,322.229 186.419,306.449 187.74,283.116 189.267,280.052 190.873,290.204 192.483,270.51 193.467,295.838 195.388,264.062 197.018,232.667 
 198.787,236.584 199.394,242.521 201.383,257.342 202.035,246.849 202.976,254.485 204.546,240.232 205.999,240.482 207.161,248.035 208.967,232.963 210.216,229.917 
 211.494,226.327 213.453,265.802 215.002,278.491 216.147,274.614 217.473,240.61 219.259,233.57 221.143,287.236 223.332,286.424 224.224,272.549 2

In [99]:
histogram(sqrt.(ηs[end÷2:end]), nbins=20, label="eta")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0054 
 
 
 0.0055 
 
 
 0.0056 
 
 
 0.0057 
 
 
 0.0058 
 
 
 0.0059 
 
 
 0.0060 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 1200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 eta

## References

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility estimation. [arxiv:1801.09956](https://arxiv.org/abs/1801.09956), 2018.

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility learning under microstructure noise. In preparation.